In [ ]:
import pandas as pd
# Dataset will be stored in a Pandas Dataframe

url = 'https://raw.githubusercontent.com/DataScience-in-Tourism/Chapter-15-Introduction-Natural-Language-Processing/main/travelsomeday.csv'
df = pd.read_csv(url)
df.head(1)

,,,,,,,,,,,,,,,,,,,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18
1,https://twitter.com/_LoveCaribbean/statuses/1252926931634458624,https://twitter.com/_LoveCaribbean/statuses/1252926931634458624,2020-04-22 07:47:13,_LoveCaribbean,RT @grenadaexplorer: A tropical treasure to cherish forever 🌅 For exclusive #TravelTips and island insights visit https://t.co/wFawu512ad #…,A tropical treasure to cherish forever 🌅 For exclusive #TravelTips and island insights visit https://t.co/wFawu512ad #Grenada #TimeToLime #PureGrenada #SpiceIsle #ILoveGrenada #DreamTodayTravelTomorrow #TravelSomeDay #WeBelieveInTravel #GrenadaDreaming #CaribbeanDreaming https://t.co/64Kg2mPH3l,Twitter Web App,1356684936,https://pbs.twimg.com/profile_images/474150473876770817/aNbE9lsf_normal.jpeg,8047,1099,3019,2013-04-16 07:55:23,#CaribbeanDreaming The diversity of the Caribbean makes it a wonderful holiday destination whatever your tastes or interests.,London,en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6027 entries, (1, 'https://twitter.com/_LoveCaribbean/statuses/1252926931634458624', 'https://twitter.com/_LoveCaribbean/statuses/1252926931634458624', '2020-04-22 07:47:13', '_LoveCaribbean', 'RT @grenadaexplorer: A tropical treasure to cherish forever 🌅 For exclusive #TravelTips and island insights visit https://t.co/wFawu512ad #…', 'A tropical treasure to cherish forever 🌅 For exclusive #TravelTips and island insights visit https://t.co/wFawu512ad #Grenada #TimeToLime #PureGrenada #SpiceIsle #ILoveGrenada #DreamTodayTravelTomorrow #TravelSomeDay #WeBelieveInTravel #GrenadaDreaming #CaribbeanDreaming https://t.co/64Kg2mPH3l', 'Twitter Web App', 1356684936, 'https://pbs.twimg.com/profile_images/474150473876770817/aNbE9lsf_normal.jpeg', 8047, 1099, 3019, '2013-04-16 07:55:23', '#CaribbeanDreaming The diversity of the Caribbean makes it a wonderful holiday destination whatever your tastes or interests.', 'London', 'en', nan, nan) to (602

As we can see from here, the text data is at the index. We need to convert indexes to columns.

In order to do this, first we need to give a name to columns in the index

In [ ]:
df= df.rename_axis(['id1','url','url2','date','username', 'quote', 'tweet','as1','as2','as3','as4','as5','as6','as7','as8','as9','as10','as11','a12'])
df.columns.to_list()

['id',
 'guid',
 'link',
 'pubdate',
 'author',
 'title',
 'description',
 'source',
 'user_id',
 'profile_image_url',
 'user_statuses_count',
 'user_friends_count',
 'user_followers_count',
 'user_created_at',
 'user_bio',
 'user_location',
 'lang',
 'coords',
 'Unnamed: 18']

In [ ]:
df.sample(1)

,,,,,,,,,,,,,,,,,,,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18
id1,url,url2,date,username,quote,tweet,as1,as2,as3,as4,as5,as6,as7,as8,as9,as10,as11,a12,,,,,,,,,,,,,,,,,,,
4441,https://twitter.com/clairesturz/statuses/1266648890331316225,https://twitter.com/clairesturz/statuses/1266648890331316225,2020-05-30 04:33:23,clairesturz,The Best Things to do in #Guatemala #travelsomeday #armchairtravel #plannowtravellater https://t.co/8mcZIhZpn2,The Best Things to do in #Guatemala #travelsomeday #armchairtravel #plannowtravellater https://t.co/8mcZIhZpn2,Revive Social App,874023301,https://pbs.twimg.com/profile_images/613107654643351552/43F_WT1I_normal.jpg,41435,19558,24768,2012-10-11 13:32:59,Helping you plan your best future travel adventure by sharing my tried & tested budget travel tips. #travelsomeday #armchairtravel #dreamtodaytravellater,"England, United Kingdom",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.reset_index(inplace=True)
df.sample(1)

,id1,url,url2,date,username,quote,tweet,as1,as2,as3,as4,as5,as6,as7,as8,as9,as10,as11,a12,id,guid,link,pubdate,author,title,description,source,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,lang,coords,Unnamed: 18
5115,5116,https://twitter.com/clairesturz/statuses/12701...,https://twitter.com/clairesturz/statuses/12701...,2020-06-08 22:10:57,clairesturz,Travel Planning Made Easy: Essential Travel Re...,Travel Planning Made Easy: Essential Travel Re...,Revive Social App,874023301,https://pbs.twimg.com/profile_images/613107654...,41870,19555,24753,2012-10-11 13:32:59,Helping you plan your best future travel adven...,"England, United Kingdom",en,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.tweet.sample(5).to_list()

['Top 10 Sights to See in Nuremberg Old Town, Germany https://t.co/jXbkI4WjiL #travelsomeday #wanderlust #armchairtravel https://t.co/msP14ISJ5r',
 'CDC Extends Its Ban on Cruising to July 19 (with some exceptions): https://t.co/BD83s859He #CoronaVirus #TravelSomeday https://t.co/mgyYMruNAL',
 '#FutureTravel #TravelAgain #VirtualTravel #TravelInspiration #TravelPlans #TravelLater #TravelSomeday #Savetourism #travelbloggers #travellovers #puravida Eco-vacation in Costa Rica: https://t.co/VBNIBh8zdl',
 "Your daily COVID-19 travel update &gt; https://t.co/QEMh64g49P  Where\\'s opening up? Where\\'s still closed? And who is permitted to go where, and when?  #CoronavirusLockdown #TravelSomeday https://t.co/vwvwyGis3s",
 'Top Things to do in San Juan del Sur, #Nicaragua #travelsomeday #armchairtravel #staythefhome https://t.co/LrZaMIAA8e #diving']

In [ ]:
#We have two columns as a text. Let's check them and decide which one we will work on
df.loc[:,['quote', 'tweet']].sample(10)

,quote,tweet
1063,RT @always5star: Happy #EarthDay everyone! Her...,Happy #EarthDay everyone! Here are a few of my...
4936,"Free Things to do in Vancouver, #Canada #trave...","Free Things to do in Vancouver, #Canada #trave..."
4976,Eating #Prague Food Tour Review #travelsomeday...,Eating #Prague Food Tour Review #travelsomeday...
2396,RT @clairesturz: Why You Should Visit #Prague ...,Why You Should Visit #Prague in Winter #travel...
1289,How to Survive Spending #Christmas Abroad #tra...,How to Survive Spending #Christmas Abroad #tra...
1911,RT @RoarLoudTravel: A66 I want to go back to C...,"A66 I want to go back to California, it has be..."
1120,RT @always5star: Happy #EarthDay everyone! Her...,Happy #EarthDay everyone! Here are a few of my...
674,RT @Frommers: Advice &amp; a warning: https://...,Advice &amp; a warning: https://t.co/0fkrgZJ0h...
3848,Things to do in Tayrona National Park: Beaches...,Things to do in Tayrona National Park: Beaches...
4998,11 Ways to Reduce the Carbon Footprint of Your...,11 Ways to Reduce the Carbon Footprint of Your...


It looks like quote column is mostly retweets. I will go with ***tweet*** column from now on, and apply text cleaning on it.

# **Text Cleaning**

In this step, these text cleaning steps will be applied on tweets:



*   Make all text lower case
*   Expanding Contractions
*   Removal of Stop Words
*   Removing URLs
*   Remove html HTML tags
*   Removing Emojis
*   Remove Emoticons
*   Stemming & Lemmatization (and the difference)
*   Spell Correction
*   Normalization
*   Removing Punctuation
*   TF-IDF



In [ ]:
!pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Here are the feature enginnering scripts that can be done before data cleaning

# Number of stopwords
stop = stopwords.words('english')
df['stopwords'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))

#Number of punctuation
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['tweet'].apply(lambda x: count_punct(x))

# Number of hashtag characters
df['hastags'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

# Number of numerics
df['numerics'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# Number of Uppercase words
df['upper'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

# sentence count
dtf['sentence_count'] = df["tweet"].apply(lambda x: len(str(x).split(".")))


### **Make all text lower case**

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['tweet'].head()

0    a tropical treasure to cherish forever 🌅 for e...
1    backpacking in #cuba on a budget #travelsomeda...
2    we likeee! https://t.co/2g1hmcpsvn #deltaairli...
3    experience buenos aires like a local #travelso...
4    who can\'t wait for grenada sailing festival 2...
Name: tweet, dtype: object

Before going further, let's extract number of punctuation for each tweet as a new feature


In [ ]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count
#Apply the defined function on the text data
df['punctuation'] = df['tweet'].apply(lambda x: count_punct(x))
#Let's check the dataset
df[['tweet','punctuation']].head()

,tweet,punctuation
0,a tropical treasure to cherish forever 🌅 for e...,21
1,backpacking in #cuba on a budget #travelsomeda...,9
2,we likeee! https://t.co/2g1hmcpsvn #deltaairli...,9
3,experience buenos aires like a local #travelso...,9
4,who can\'t wait for grenada sailing festival 2...,25


In [ ]:
# Additionally, let's save number of hastags are new features
df['hastags'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['tweet','hastags']].head()

,tweet,hastags
0,a tropical treasure to cherish forever 🌅 for e...,11
1,backpacking in #cuba on a budget #travelsomeda...,4
2,we likeee! https://t.co/2g1hmcpsvn #deltaairli...,3
3,experience buenos aires like a local #travelso...,4
4,who can\'t wait for grenada sailing festival 2...,11


### **Expand Contractions**

Contractions are words or combinations of words which are shortened by dropping letters and replacing them by an apostrophe. 

Let’s have a look at some examples:
* we’re = we are
* we’ve = we have
* I’d = I would

Note: This step needs to be done before word tokenizer because NLTK word tokenizer has in-built methods for dealing with contractions. However, NLTK word tokenizer's approach separates contractions without expanding. Expanding is a better method than simply separating.

In [ ]:
!pip install contractions

In [ ]:
import contractions

# Example text 
text = ''' She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.'''
  
# creating an empty list 
expanded_words = []     
for word in text.split(): 
  # using contractions.fix to expand the shotened words 
  expanded_words.append(contractions.fix(word))    
    
expanded_text = ' '.join(expanded_words) 
print('Original text: ' + text) 
print('\n') 
print('Expanded_text: ' + expanded_text)

Original text:  She'll be airport in 30 mins. We are supposed to catch the arrival, aren't we?  
          I'd love to welcome her personally. It'll be an awesome vacation.


Expanded_text: she will be airport in 30 mins. We are supposed to catch the arrival, are not we? I would love to welcome her personally. it will be an awesome vacation.


In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: contractions.fix(x))
df.tweet.sample(5)

3281    how the visitor experience in #hawaii will cha...
1838    nine reasons to visit the north of #england #t...
5787    an #edinburgh food tour with eat walk edinburg...
1198    if you love hostels, now is the time to #adopt...
4301    taste #belize food tours: the best of belize c...
Name: tweet, dtype: object

### **Removal of Stop Words** 

In [ ]:
# before removing stopwords, we can count count them.
# Remember that we already did this in the beginning. But it could be done here as well.
df['stopwords'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['tweet','stopwords']].head()

,tweet,stopwords
0,a tropical treasure to cherish forever 🌅 for e...,4
1,backpacking in #cuba on a budget #travelsomeda...,3
2,we likeee! https://t.co/2g1hmcpsvn #deltaairli...,1
3,experience buenos aires like a local #travelso...,1
4,who can\'t wait for grenada sailing festival 2...,3


In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['tweet'].sample(10)

5332    best #hostel #lisbon #portugal solo travellers...
2382    blarney castle tour &amp; kissing blarney ston...
2574    secret food tour #london #travelsomeday #armch...
2587    big changes coming #hawaii tourist experience ...
2066    get #cusco #ollantaytambo &amp; go! #travelsom...
2365    ireland announces 5-step plan reopening, makes...
55      free walking tour #mexico city: estación mexic...
3885    improve travel photography can\'t actually tra...
2881    #hiking live #volcano #antigua #guatemala #tra...
939     pauline frommer (author frommer guides #nyc) w...
Name: tweet, dtype: object

#### **Adding common words from our document to stop_words**

Doing this step can be better after TF-IDF because TF-IDF will also remove many words. Or, at the end of the data cleaning, most frequent words can be checked. If we think that there are some irrelevant or meaningless common words, we can remove them by adding as new stopwords.



In [ ]:
add_words = ["also","im","ive",]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['tweet'].sample(10)

4565    top things see &amp; #portugal https://t.co/vj...
1474    wow, knew #ireland beautiful, idea stunning ge...
1421    travel crafts? now\'s time! @herpackinglist in...
1390    anyone tried stand paddleboarding? think looks...
3015    try visit #venice day – need one #travelsomeda...
3613    best hikes peru – hikes &amp; treks #peru #tra...
3255    reflecting previous #travel - you\'ve visited ...
5998    sure start comes getting back traveling planni...
4772    stairway gods san miguelito #cancún. 🇲🇽 @museo...
3682    good idea keep logbook? ⛵️ 📙 info: https://t.c...
Name: tweet, dtype: object

### **Remove URLs** 

In [ ]:
import re

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

df['tweet'] = df.tweet.apply(remove_urls)
df.tweet.head()

0    a tropical treasure to cherish forever 🌅 for e...
1    backpacking in #cuba on a budget #travelsomeda...
2    we likeee!  #deltaairlines #coronavirus #trave...
3    experience buenos aires like a local #travelso...
4    who can\'t wait for grenada sailing festival 2...
Name: tweet, dtype: object

In [ ]:
# Example
sentence=  'NPR can provide useful information https://www.npr.org'
remove_urls(sentence)

'NPR can provide useful information '

### **Remove html tags** 

In [ ]:
from bs4 import BeautifulSoup

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

#Example
strip_html_tags('People and neighbors are for the most part very friendly,</a> even in stores</td>')

'People and neighbors are for the most part very friendly, even in stores'

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: strip_html_tags(x))
df['tweet'].head()

0    tropical treasure cherish forever 🌅 exclusive ...
1    backpacking #cuba budget #travelsomeday #armch...
2    likeee!  #deltaairlines #coronavirus #travelso...
3    experience buenos aires like local #travelsome...
4    can\'t wait grenada sailing festival 2021?! ☝️...
Name: tweet, dtype: object

###**Removing Emojis**

Eventhough emojis can be indictor of some emotions that can be related to the tweet, we need to remove the emojis in our text analysis because for now we don't have a strong method to utulize emojis.

In [ ]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

#Example
remove_emoji("Have fun with NLP! 😃😃")

'Have fun with NLP! '

In [ ]:
# remove all emojis from tweets
df['tweet'] = df['tweet'].apply(lambda x: remove_emoji(x))

### **Remove Emoticons**

In previous steps, we have removed emoji. Now, going to remove emoticons.

What is the difference between emoji and emoticons?

:-) is an emoticon
😜 → emoji.

In [ ]:
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [ ]:
#Example
remove_emoticons("I used to play Super Mario games :)")

'I used to play Super Mario games '

In [ ]:
df['tweet'] = df['tweet'].apply(lambda x: remove_emoticons(x))
df.tweet.sample(5)

1614    my latest for @fodorstravel, all about what ho...
2001    how to date a woman who travels #travelsomeday...
4016    9 things to know before visiting the country o...
4756    cairo is one of the most amazing cities in the...
4858    the top 10 things to see &amp; do in #portugal...
Name: tweet, dtype: object

### **Stemming and Lemmatization**

**Definition**: Stemming usually refers to a crude heuristic process that chops off the ends of words in the hope of achieving this goal correctly most of the time, and often includes the removal of derivational affixes. Lemmatization usually refers to doing things properly with the use of a vocabulary and morphological analysis of words, normally aiming to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma . 

Source: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html



In [ ]:
#Example (adapted from datacamp)
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
porter=PorterStemmer()

In [ ]:
#Tourism specific words to be stemmed and lemmatized
word_list = ["tourist", "booking", "rating","itinerary","recreation","amenities","attractions","sightseeing","eating"]
print("{0:20}{1:20}{2:20}".format("Original Word","Stemmed","Lemmatized"))

for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,porter.stem(word),wordnet_lemmatizer.lemmatize(word,pos="v")))

Original Word       Stemmed             Lemmatized          
tourist             tourist             tourist             
booking             book                book                
rating              rate                rat                 
itinerary           itinerari           itinerary           
recreation          recreat             recreation          
amenities           amen                amenities           
attractions         attract             attractions         
sightseeing         sightse             sightsee            
eating              eat                 eat                 


In [ ]:
#We can apply just one of the stemming or lemmatizion. For this study, we will go with lemmatization.
#Still here is the codes for stemming
'''
porter=PorterStemmer()

def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

df['tweet'] = df['tweet'].apply(lambda x: stemSentence(x))
df.sample(5)
'''

'\nporter=PorterStemmer()\n\ndef stemSentence(sentence):\n    token_words=word_tokenize(sentence)\n    token_words\n    stem_sentence=[]\n    for word in token_words:\n        stem_sentence.append(porter.stem(word))\n        stem_sentence.append(" ")\n    return "".join(stem_sentence)\n\ndf[\'tweet\'] = df[\'tweet\'].apply(lambda x: stemSentence(x))\ndf.sample(5)\n'

In [ ]:
#Lemmatization

def LemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    lemma_sentence=[]
    for word in token_words:
        lemma_sentence.append(wordnet_lemmatizer.lemmatize(word, pos= 'v'))
        lemma_sentence.append(" ")
    return "".join(lemma_sentence)

df['tweet'] = df['tweet'].apply(lambda x: LemmaSentence(x))
df.tweet.sample(5)

4611    ten travel tip # cuba # travelsomeday # armcha...
2544    plan trip hawaii - momma go travel # travel # ...
4695    10 planet\ 's unusual house # travelsomeday # ...
5528    2 days # edinburgh itinerary : perfect weekend...
1345    quebecois twilight micmac mountains . @ touris...
Name: tweet, dtype: object

In [ ]:
df.tweet.dtype

dtype('O')

### **Spell Correction** 
*Reminder: below explanation was taken from somewhere and needs to be parapharased*


We’ve all seen tweets with a plethora of spelling mistakes. Our timelines are often filled with hastly sent tweets that are barely legible at times.

In that regard, spelling correction is a useful pre-processing step because this also will help us in reducing multiple copies of words. For example, “Analytics” and “analytcs” will be treated as different words even if they are used in the same sense.

To achieve this we will use the textblob library.

In [ ]:
from textblob import TextBlob
#Example sentence
sentence = TextBlob("I havv goood speling!")
print(sentence.correct())

I have good spelling!


In [ ]:
# Example sentence related to tourism context
print(TextBlob('In Chicago, a lakeside is a goodf lcation to relax').correct())
print(TextBlob('Minesota is briming with natual and cltural bauty').correct()) 

In Chicago, a lakeside is a good location to relax
Minnesota is brimming with natural and cultural beauty


In [ ]:
df.tweet[150:200].to_list()

['an active holiday in spain: cool things to do in mojácar #travelsomeday #armchairtravel #staythefhome  #almeria',
 'if you love hostels, now is the time to #adoptahostel! you can give struggling hostels a lifeline by buying a voucher to use on a future stay at  #travelsomeday #plannowtravellater with @budgettraveller @hostelexperts ',
 'nantucket dreams #travelsomeday ',
 'big island dreams. #travelsomeday ',
 'virtually travel and explore new zealand in this article from guest writer &amp; photographer courtney ley &gt;&gt;  #stayhome #travelsomeday #dreamnowtravellater ',
 'what is the boleto turistico #cusco tourist ticket &amp; why do you #travelsomeday #armchairtravel #staythefhome  #peru',
 'if you love hostels, now is the time to #adoptahostel! you can give struggling hostels a lifeline by buying a voucher to use on a future stay at  #travelsomeday #plannowtravellater with @budgettraveller @hostelexperts ',
 'sao paulo street art #travelsomeday #armchairtravel #staythefhome  #

In [ ]:
%time v.apply(lambda x: str(TextBlob(x).correct()))

CPU times: user 21.2 s, sys: 104 ms, total: 21.3 s
Wall time: 21.4 s


0    tropical treasure cherish forever exclusive # ...
1    backpack # cuba budget # travelsomeday # armch...
2    like ! # deltaairlines # coronavirus # travels...
3    experience burns air like local # travelsomeda...
4    can\'t wait grenade sail festival 2021 ? ! exc...
Name: tweet, dtype: object

In [ ]:
df.tweet.head()

0    tropical treasure cherish forever exclusive # ...
1    backpack # cuba budget # travelsomeday # armch...
2    likeee ! # deltaairlines # coronavirus # trave...
3    experience buenos air like local # travelsomed...
4    can\'t wait grenada sail festival 2021 ? ! exc...
Name: tweet, dtype: object

In [ ]:
#This takes a bit long to complete. I will skip for now.

from textblob import TextBlob
%time df['tweet'] = df['tweet'].apply(lambda x: str(TextBlob(x).correct()))
print(df.tweet.head())

### **Removing Punctuation**

A reminder: if you remove punctuation earlier then html tags and URLs, then it won't be possible to remove them probably later. That's why, removing punctuation the last is a wise choice.


In [ ]:
df['tweet'] = df['tweet'].str.replace('[^\w\s]','')
df['tweet'].head()

0    tropical treasure cherish forever exclusive  t...
1    backpack  cuba budget  travelsomeday  armchair...
2    like   deltaairlines  coronavirus  travelsomeday 
3    experience burns air like local  travelsomeday...
4    cant wait grenade sail festival 2021   exclusi...
Name: tweet, dtype: object

In [ ]:
# Example
sentence = "Do you listen to music a lot?"
re.sub(r'[^\w\s]','',sentence)

'Do you listen to music a lot'

### **Language Detection**


In [ ]:
import os, re, nltk, spacy, string, umap

pd.set_option('display.max_colwidth', 50) 
nlp = spacy.load('en_core_web_sm')

!pip install spacy-langdetect
from spacy_langdetect import LanguageDetector
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

     |████████████████████████████████| 1.0MB 4.3MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp37-none-any.whl size=993460 sha256=35d43eef096d5d626f1ccea9d353094ef4ef9244020020e19332d13e2377dc78
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [ ]:
def detect_language(tweet):
    try:
        doc = nlp(t)
        language = doc._.language['language']
        score = doc._.language['score']
    except:
        language = ''
        print('sth goes wrong')
    return language, score

languages = []
scores = []
for i, t in df['tweet'].iteritems():
    l, s = detect_language(t)
    if i % 100 ==0:
        print(i, t, l)  
    languages.append(l)
    scores.append(s)
    
df['lng'] = languages
df['lng_score'] = scores

0 tropical treasure cherish forever exclusive  traveltips island insight visit  grenade  timetolime  puregrenada  spiceisle  ilovegrenada  dreamtodaytraveltomorrow  travelsomeday  webelieveintravel  grenadadreaming  caribbeandreaming  en
100 charlie chocolate factory afternoon tea  london  travelsomeday  armchairtravel  staythefhome  england  en
200 nuestros compañeros de travel est abriendo frontiers captando naevus ideas para el tourism de nuestra provincial     stop chaos son nos verdaderos cracks   tourism  evil  travelsomeday  es
300  travelsomeday  no
400 virtually travel explore new zealand article guest writer  photographer journey let    stayhome  travelsomeday  dreamnowtravellater  en
500 sometimes sasha draconine  othertimes look like this   thailandinsider  thailandfanclub  gothaibefree  thailand  bangkok  watprakaew  we  tt  trot  travel  travelblogger  solotravel  gaytravel  ilovelgbttravel  travelfromhome  travelsomeday  armchairtravel  picoftheday  en
600 wonder  hokusa

In [ ]:
df[['tweet', 'lng','lng_score']].sample(10).round(2)

,tweet,lng,lng_score
2497,lavender field an river moles writes outdoor...,en,1.00
4359,djurönäset relax stockholm archipelago tra...,sv,0.86
1162,friends beat page travel include wichita s ke...,en,1.00
2521,2 days edinburgh itinerant perfect weekend ...,en,1.00
4807,manage attitude custom attitude sickness much...,en,1.00
5134,xavc67 many case you right that s include has...,en,1.00
5036,eat venice traditional food venice try trav...,en,1.00
2175,visit frederiksborg castle hillerød denmark ...,da,1.00
5264,like simon mountains ethiopia need add anyone...,no,0.57
2255,climb of arena nine partner of travelsomeday...,en,1.00


In [ ]:
df.lng.value_counts()

en    5607
no     186
it      76
fr      38
sv      35
da      33
ca      26
es      20
id       2
nl       1
hr       1
el       1
sl       1
Name: lng, dtype: int64

In [ ]:
language_counts = df.lng.value_counts()
pd.DataFrame(language_counts)

'no' stands for Norwegian language. 

For more info about acronyms of languages:
https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes

In [ ]:
# Number of English Tweets
df.loc[df.lng == 'en'].shape

(5607, 43)

In [ ]:
# Keep only English language and drop the rest
df = df.loc[df.lng == 'en']

### **TF_IDF**


In [ ]:
#import the TfidfVectorizer from Scikit-Learn.
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=.80, min_df=5, stop_words=None, use_idf=True, norm=None)
transformed_tweets = vectorizer.fit_transform(df.tweet)

print(transformed_tweets.shape)

In [ ]:
dense = transformed_tweets.todense()
denselist = dense.tolist()
feature_names = vectorizer.get_feature_names()
df2 = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
df2.head()

In [ ]:
transformed_tweets_as_array = transformed_tweets.toarray()
# use this line of code to verify that the numpy array represents the same number of documents that we have in the file list
len(transformed_tweets_as_array)

### **Tokenization**


For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.



In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
data_cv = cv.fit_transform(df.tweet)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
data_dtm


In [ ]:
cv = CountVectorizer(max_df = 0.90, min_df= 0.001)
data_cv = cv.fit_transform(df.tweet)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names()) #Document term matrix
data_dtm.index = df.index
data_dtm

In [ ]:
# Let's pickle it for later use
data_dtm.to_pickle("cleaned_tweets.pkl")

#Notice that the pickled data contains only tweets, not other columns.



**What Can You Do With pickle?**

Pickling is useful for applications where you need some degree of persistency in your data. 
Your program's state data can be saved to disk, so you can continue working on it later on.
It can also be used to send data over a Transmission Control Protocol (TCP) or 
socket connection, or to store python objects in a database. Pickle is very 
useful for when you're working with machine learning algorithms, where you want
to save them to be able to make new predictions at a later time, without
having to rewrite everything or train the model all over again.

Source:https://www.datacamp.com/community/tutorials/pickle-python-tutorial#whatfor




---
# **FEATURE EXTRACTION**

For the feature extraction, we will apply following steps in order. Depending on your study and data, you may skip some of these features and add your own feature extractions.

 word count: counts the number of tokens in the text (separated by a space)

1.   **character count**: 
2. **average word length**


Features that can only be obtained before the text cleaning


* Number of stop words
* The number of punctuation
* Number of hashtag characters
* Number of numerical characters
* Number of Uppercase words
* sentence count

In the book, as an egzersize, it can be asked: Why these features can only be obtained before text cleaning? Explain for each of them.



In [ ]:
# Number of hashtag characters
df['hastags'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

#character count
df['char_count'] = df['tweet'].str.len() ## this also includes spaces

def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

df['avg_word'] = df['tweet'].apply(lambda x: avg_word(x)).round(1)

df[['tweet','char_count','avg_word']].sample(5)

Keep in mind that in feature enginnering process, we are not changing the text (the tweet) column. Therefore, the pickled file stays unchanged.

In [ ]:
#Let's save cleaned data

df.to_csv('tourism_tweets_cleaned.csv', index=False)

### **Checking requirements for this notebook**

In [ ]:
!pip3 freeze > requirements.txt

In [ ]:
# show the file's contents
! cat requirements.txt

In [ ]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

# Print the type of the returned object
print("os.getcwd() returns an object of type: {0}".format(type(cwd)))

Current working directory: /content
os.getcwd() returns an object of type: <class 'str'>
